In [1]:
import librosa
import time
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    for file in glob.glob("C:/Users/SHIVA/OneDrive/Desktop/TESS/TESS Toronto emotional speech set data/*AF_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=file_name.split("_")[2][:-4]
        if emotion == 'ps':
            emotion = 'surprised'
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    
    return train_test_split(np.array(x), y, test_size=test_size, random_state=0)

In [5]:
#Split the dataset
#and sampling rate
start_time = time.time()
x_train,x_test,y_train,y_test=load_data(test_size=0.2)
print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

--- Data loaded. Loading time: 78.65515637397766 seconds ---


In [6]:
samplingrate=5000//60
print(samplingrate)

83


In [7]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1094, 274)


In [8]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [9]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [10]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [11]:
#Predict for the test set
y_pred=model.predict(x_test)

In [12]:
y_pred

array(['happy', 'happy', 'fearful', 'disgust', 'happy', 'happy', 'happy',
       'happy', 'disgust', 'happy', 'happy', 'happy', 'happy', 'disgust',
       'calm', 'happy', 'fearful', 'happy', 'disgust', 'disgust',
       'disgust', 'happy', 'happy', 'happy', 'happy', 'happy', 'disgust',
       'happy', 'calm', 'disgust', 'happy', 'disgust', 'happy', 'happy',
       'happy', 'calm', 'disgust', 'disgust', 'happy', 'fearful',
       'disgust', 'happy', 'happy', 'fearful', 'calm', 'happy', 'happy',
       'calm', 'disgust', 'disgust', 'happy', 'disgust', 'calm',
       'disgust', 'happy', 'disgust', 'fearful', 'disgust', 'happy',
       'happy', 'disgust', 'calm', 'disgust', 'happy', 'happy', 'calm',
       'calm', 'fearful', 'disgust', 'calm', 'disgust', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'disgust', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'fearful', 'calm', 'disgust',
       'happy', 'calm', 'happy', 'happy', 'disgust', 'disgust', 'disgust',


In [13]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 82.12%


In [14]:
from sklearn.metrics import accuracy_score, f1_score

In [15]:
f1_score(y_test, y_pred,average=None)

array([0.81578947, 0.81879195, 0.49056604, 0.88888889])

In [16]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,happy,happy
2,fearful,fearful
3,disgust,disgust
4,happy,happy
5,happy,happy
6,disgust,happy
7,happy,happy
8,happy,disgust
9,happy,happy


In [17]:
import pickle
# Writing different model files to file
with open( 'emotion.sav', 'wb') as f:
    pickle.dump(model,f)

In [18]:
filename = 'emotion.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage
audio='Actor_01/03-01-01-01-01-01-01.wav'
feature=extract_feature(audio, mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction[0]

'happy'